### Строим логистическую регрессию - уровень дохода

In [4]:
import pandas as pd
import numpy as np

In [5]:
#Скачиваем данные из файла
data = pd.read_csv( 'adult.csv' )
data.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [6]:
#Смотрим информацию о данных. В данном датасете пропусков нет, обработка NaN не требуется
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
age                48842 non-null int64
workclass          48842 non-null object
fnlwgt             48842 non-null int64
education          48842 non-null object
educational-num    48842 non-null int64
marital-status     48842 non-null object
occupation         48842 non-null object
relationship       48842 non-null object
race               48842 non-null object
gender             48842 non-null object
capital-gain       48842 non-null int64
capital-loss       48842 non-null int64
hours-per-week     48842 non-null int64
native-country     48842 non-null object
income             48842 non-null object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


In [7]:
from sklearn.linear_model import LogisticRegression

In [8]:
# целевая переменная (столбец income) является категориальной
# переведем значения столбца в числа, оставив один столбец

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [11]:
le.fit( data['income'] )

LabelEncoder()

In [12]:
# записываем в переменную y преобразованный столбец income

y = pd.Series( data = le.transform( data['income'] ) )
y.head()

0    0
1    0
2    1
3    1
4    0
dtype: int64

In [13]:
# попробуем выбрать какие-нибудь признаки
selectedColumns = data[ [ 'workclass', 'educational-num', 'relationship' ] ]

# столбцы 'workclass', 'relationship' является категориальными
# переведем их в значения 0 и 1, добавив столбцы с соответствующими названиями
X = pd.get_dummies( selectedColumns, columns = [ 'workclass', 'relationship' ] )

X.head()

,educational-num,workclass_?,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay,relationship_Husband,relationship_Not-in-family,relationship_Other-relative,relationship_Own-child,relationship_Unmarried,relationship_Wife
0,7,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
1,9,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0
2,12,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
3,10,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0
4,10,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [15]:
#выбираем модель
model = LogisticRegression()

In [16]:
# обучаем модель
model.fit( X, y )
predictions = model.predict_proba( X )

In [17]:
predictions[:5]

array([[ 0.99516889,  0.00483111],
       [ 0.67814189,  0.32185811],
       [ 0.41541903,  0.58458097],
       [ 0.59227678,  0.40772322],
       [ 0.99350129,  0.00649871]])

In [18]:
# сравниваем факт с предсказаниями
# (да, сравнивать предсказания с самими данными не круто, лучше было разбить на обучающую и тестовую выборку)
# это будет в следующем занятии

zip( predictions[:, 1], y )

In [19]:
model.score(X, y)

0.82476147577904257

### Выбираем тройку лучших параметров

In [20]:
#Делаем массив из названий всех столбцов
cols = data.columns.values
cols

array(['age', 'workclass', 'fnlwgt', 'education', 'educational-num',
       'marital-status', 'occupation', 'relationship', 'race', 'gender',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'income'], dtype=object)

In [21]:
#Делаем массив названий непрерывных столбцов
continuous_cols=['age', 'fnlwgt', 'educational-num', 'capital-gain', 'capital-loss', 'hours-per-week']

In [22]:
#Делаем массив названий столбцов с перечислимым набором значений
nominal_cols=['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'native-country']

In [23]:
#Запоминаем длинну выборки столбцов - 1 (последний столбец - целевая переменная)
l=cols.size - 1

#Заготовка под массив столбцов
c =[]

#Заготовка под массив столбцов перечислимых типов
nc = []

#Переменная под максимальное значение score
maxValue=0

#В трех циклах последовательно проходим по столбцам и формируем тройки, для каждой трокий делаем get_dummies для 
#перечислимых переменных, строим модель и вычисляем score.
#В переменных maxValue и bestParams сохраняем значения лучщего score и набор параметров, для которого он получился
for i in range(0, l, 1):
    for j in range(i+1, l, 1):
        for k in range(j+1, l, 1):
            #Проверяем, что у нас нет одинаковых столбцов в выборке
            if cols[i]==cols[j] or cols[i]==cols[k] or cols[j]==cols[k]:
                break
                
            #Добавляем названия столбцов в выборку    
            c.append(cols[i])
            c.append(cols[j])
            c.append(cols[k])
            
            #Проверяем наличие перечислимых столбцов
            if cols[i] in nominal_cols:
                nc.append(cols[i])
            if cols[j] in nominal_cols:
                nc.append(cols[j])
            if cols[k] in nominal_cols:
                nc.append(cols[k])
            
            #Выделяем столбцы для эксперимента
            selectedColumns = data[[i for i in c]]
            cls=[i for i in nc]
            
           #Разделяем столбцы с перечислимыми переменными из выборки
            X = pd.get_dummies( selectedColumns, columns = cls)
            model = LogisticRegression()
            
            #Обучаем модель
            model.fit( X, y )
            
            #Считаем score
            ms=model.score(X, y)
            
            #Проверяем, не является ли score максимальным
            if ms>maxValue:
                maxValue=ms
                bestParams=c
                       
            c=[]
            nc=[]
            
#Выводим лучшие параметры            
print('Лучший набор параметров: ', bestParams)
print('Значение score: ', maxValue)

Лучшай набор параметров:  ['education', 'relationship', 'capital-gain']
Значение score:  0.838724867942
